# Evaluating Models

This notebook contains the code used for evaluating the following learning models:

-  **Standard GBDT** (_baseline 1_)
-  **Adversarial Boosting** (_baseline 2_)
-  **Non-Interferent GBDT** (our proposal)

# Documentation

 - http://lightgbm.readthedocs.io/en/latest/
 - http://lightgbm.readthedocs.io/en/latest/Python-Intro.html
 - https://github.com/Microsoft/LightGBM/tree/master/examples/python-guide

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import json
import glob
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm
import functools
from os import listdir
from os.path import isfile, join
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix

from NILib import *
# Adding the following line, allows Jupyter Notebook to visualize plots
# produced by matplotlib directly below the code cell which generated those.
%matplotlib inline

np.random.seed(73)

# Standard evaluation metric

The following function is the one used for evaluating the quality of the learned model (either _standard_, _adversarial-boosting_, or _non-interferent_). This is the standard <code>avg_log_loss</code>.

In [ ]:
def logistic(x):
    return 1.0/(1.0 + np.exp(-x))

In [ ]:
def logit(p):
    return np.log(p/(1-p))

# <code>avg_log_loss</code>

In [ ]:
# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
def avg_log_loss(preds, train_data):
    
    labels = train_data.get_label()
    losses = np.log(1.0 + np.exp(-preds*labels))
    avg_loss = np.mean(losses)
    
    return 'avg_binary_log_loss', avg_loss, False

In [ ]:
def eval_log_loss(model, test, test_groups=None, svm=False):
    
    lgbm_test = lightgbm.Dataset(data=test.iloc[:,:-1].values, 
                                 label=test.iloc[:,-1].values,
                                 free_raw_data=False)
    
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        return avg_log_loss(logit(model.predict_proba(test.iloc[:,:-1].values)[:,1]), lgbm_test)[1]
    
    return avg_log_loss(model.predict(test.iloc[:,:-1].values), lgbm_test)[1]

# Custom evaluation metric

Similarly to what we have done for <code>fobj</code>, <code>feval</code> can be computed from a weighted combination of two evaluation metrics:

-  <code>avg_log_loss</code> (standard, defined above);
-  <code>avg_log_loss_uma</code> (custom, defined below).

# <code>avg_log_loss_uma</code>

This is the binary log loss yet modified to operate on groups of perturbed instances.

In [ ]:
# Our custom metric

def binary_log_loss(pred, true_label):

    return np.log(1.0 + np.exp(-pred * true_label))

# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
def avg_log_loss_uma(preds, train_data):
    labels = train_data.get_label()
    attack_lens = train_data.get_group()
    
    offset = 0
    max_logloss = []
    avg_max_logloss = 0.0
    
    if attack_lens is not None:
    
        for atk in attack_lens:
            losses = [binary_log_loss(h,t) for h,t in zip(preds[offset:offset+atk], labels[offset:offset+atk])]
            max_logloss.append(max(losses))

            offset += atk
        
        avg_max_logloss = np.mean(max_logloss)  

    return 'avg_binary_log_loss_under_max_attack', avg_max_logloss, False

In [ ]:
def eval_log_loss_uma(model, test, test_groups=None, svm=False):
    
    lgbm_test = lightgbm.Dataset(data=test.iloc[:,:-1].values, 
                                 label=test.iloc[:,-1].values,
                                 group=test_groups,
                                 free_raw_data=False)
    
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        return avg_log_loss_uma(logit(model.predict_proba(test.iloc[:,:-1].values)[:,1]), 
                                               lgbm_test)[1]
    
    return avg_log_loss_uma(model.predict(test.iloc[:,:-1].values), 
                                               lgbm_test)[1]

# <code>feval=avg_non_interferent_log_loss</code>

Used for measuring the validity of any model (either _standard_, _baseline_, or _non-interferent_). More precisely, <code>avg_non_interferent_log_loss</code> is the weighted sum of the binary log loss and the binary log loss under maximal attack.

In [ ]:
# LightGBM takes lambda x,y: avg_weighted_sum_log_loss_log_loss_uma(preds, train_data, alpha=0.5)

def avg_non_interferent_log_loss(preds, train_data, alpha=1.0):
    
    # binary logloss under maximal attack
    _, loss_uma, _    = avg_log_loss_uma(preds, train_data)
    
    # binary logloss (plain)
    # _, loss_plain, _  = avg_log_loss(preds, train_data)
    
    ids = []
    attack_lens = train_data.get_group()
    
    if attack_lens is not None:
        offset=0
        for atk in attack_lens:
            ids += [offset]
            offset += atk      
            
    ids = np.array(ids)
    labels = train_data.get_label()
    losses = np.log(1.0 + np.exp(-preds[ids]*labels[ids]))
    loss_plain = np.mean(losses)

    # combine the above two losses together
    weighted_loss = alpha*loss_uma + (1.0-alpha)*loss_plain

    return 'avg_non_interferent_log_loss [alpha={:.2f}]'.format(alpha), weighted_loss, False

def eval_non_interferent_log_loss(model, test, test_groups=None, svm=False, alpha=1.0):
    
    lgbm_test = lightgbm.Dataset(data=test.iloc[:,:-1].values, 
                                 label=test.iloc[:,-1].values,
                                 group=test_groups,
                                 free_raw_data=False)
    
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        return avg_non_interferent_log_loss(logit(model.predict_proba(test.iloc[:,:-1].values)[:,1]), 
                                                  lgbm_test,
                                                  alpha=alpha
                                                 )[1]
    
    return avg_non_interferent_log_loss(model.predict(test.iloc[:,:-1].values), 
                                                  lgbm_test,
                                                  alpha=alpha
                                                 )[1]

# Additional validity measures

In addition to the evaluation metrics defined above (used for training), we also consider the following **4** measures of validity to compare the performance of each learned model:

-  <code>eval_binary_err_rate</code>: This is the traditional binary error rate (1-accuracy);
-  <code>eval_binary_err_rate_uma</code>: This is the binary error rate modified to operate on groups of perturbed instances under maximal attack.
-  <code>eval_roc_auc</code>: This is the classical ROC AUC score;
-  <code>eval_roc_auc_uma</code>: This is the ROC AUC score modified to operate on groups of perturbed instances under maximal attack.

Again, note that those are **not** metrics used at training time (i.e., they do not define any <code>feval</code>), rather they are used to assess the (offline) quality of each learned model.

# <code>eval_binary_err_rate</code>

In [ ]:
def eval_binary_err_rate(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    errs = 0
    for p,l in zip(predictions,labels):
        if p != l:
            errs += 1
    return errs/len(predictions)

# <code>eval_binary_err_rate_uma</code>

In [ ]:
def eval_binary_err_rate_uma(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    offset = 0
    errs = 0

    for g in test_groups:
        predictions_att = predictions[offset:offset+g]
        true_label = labels[offset]
        if np.any([p != true_label for p in predictions_att]):
            errs += 1
        offset += g

    return errs/len(test_groups)

# <code>eval_roc_auc</code>

In [ ]:
def eval_roc_auc(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    predictions = []
    
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        predictions = logit(model.predict_proba(X)[:,1])
    else:
        predictions = model.predict(X)
        
    
    return roc_auc_score(y_true=labels, y_score=predictions)

# <code>eval_roc_auc_uma</code>

In [ ]:
def eval_roc_auc_uma(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    predictions = []
    
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        predictions = logit(model.predict_proba(X)[:,1])
    else:
        predictions = model.predict(X)
    
    
    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        
        true_label = labels[offset]
        true_labels.append(true_label)
        predictions_att = predictions[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
    return roc_auc_score(y_true=true_labels, y_score=worst_predictions)

# <code>eval_specificity</code>

In [ ]:
def eval_specificity(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]

    tn, fp, fn, tp = confusion_matrix(labels, predictions).ravel()

    return tn/(tn + fp)

# <code>eval_specificity_uma</code>

In [ ]:
def eval_specificity_uma(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        true_label = labels[offset]
        true_labels.append(true_label)
        predictions_att = predictions[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
    tn, fp, fn, tp = confusion_matrix(true_labels, worst_predictions).ravel()

    return tn/(tn + fp)

# <code>eval_sensitivity</code>

In [ ]:
def eval_sensitivity(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]

    tn, fp, fn, tp = confusion_matrix(labels, predictions).ravel()

    return tp/(tp + fn)

# <code>eval_sensitivity_uma</code>

In [ ]:
def eval_sensitivity_uma(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        true_label = labels[offset]
        true_labels.append(true_label)
        predictions_att = predictions[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
    tn, fp, fn, tp = confusion_matrix(true_labels, worst_predictions).ravel()

    return tp/(tp + fn)

# <code>eval_f1</code>

In [ ]:
def eval_f1(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    return f1_score(y_true=labels, y_pred=predictions, average='macro')

# <code>eval_f1_uma</code>

In [ ]:
def eval_f1_uma(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        true_label = labels[offset]
        true_labels.append(true_label)
        predictions_att = predictions[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
    return f1_score(y_true=true_labels, y_pred=worst_predictions, average='macro')

# Evaluation metrics

In [ ]:
EVAL_METRICS = [eval_log_loss, 
                eval_binary_err_rate,
                eval_specificity,
                eval_sensitivity,
                eval_f1,
                eval_roc_auc
               ]

EVAL_METRICS_UNDER_MAX_ATTACK = [eval_log_loss_uma,
                                 eval_binary_err_rate_uma,
                                 eval_specificity_uma,
                                 eval_sensitivity_uma,
                                 eval_f1_uma,
                                 eval_roc_auc_uma
                                ]

# Evaluate each model w.r.t. _all_ evaluation metrics

In [ ]:
def eval_learned_model(model, eval_metric, test, test_groups=None, svm=False):
    return eval_metric(model, test, test_groups=test_groups, svm=svm)

In [ ]:
def eval_learned_models(model, model_type, test, test_groups=None, budget=0):

    eval_metrics = EVAL_METRICS
    d_test = "D_test"
    if test_groups is not None:
        eval_metrics = EVAL_METRICS_UNDER_MAX_ATTACK
        d_test = "D_test_att"
    
    header = ['Model'] + ['Budget'] + [m.__name__.replace('eval_','').replace('_',' ').title() for m in EVAL_METRICS]
    df = pd.DataFrame(columns=header)
    first_row = [model_type] + [budget] + [None for m in EVAL_METRICS]
    df.loc[0] = first_row
    
    svm = False
    if model_type == "SVM":
        svm = True

    for eval_metric in eval_metrics:
        res = eval_learned_model(model, eval_metric, test, test_groups=test_groups, svm=svm)
        print("{} learning - {} on {} = {:.5f}"
                  .format(model_type, eval_metric.__name__, d_test, res))
        column_metric = eval_metric.__name__
        if eval_metric.__name__.endswith("uma"):
            column_metric = eval_metric.__name__.replace('uma', '')
        df[column_metric.replace('eval_','').replace('_',' ').title()] = res

    print("******************************************************************************************************")
    
    return df

# Retrieve all model files

In [ ]:
DATASET_NAME="wine"
DATASET_DIR="../data/{}".format(DATASET_NAME)
MODELS_DIR="../out/models"
TRAINING_FILENAME=DATASET_DIR + "/" + "train_ori.csv.bz2"
VALIDATION_FILENAME=DATASET_DIR + "/" + "valid_ori.csv.bz2"
TEST_FILENAME=DATASET_DIR + "/" + "test_ori.csv.bz2"

In [ ]:
# load train/valid/test
train, valid, test, _ = load_atk_train_valid_test(TRAINING_FILENAME, VALIDATION_FILENAME, TEST_FILENAME)

In [ ]:
def extract_model_name(model_filename):
    model_fileroot = model_filename.split('/')[-1].split('.')[0]
    model_name = model_fileroot.split('_T')[0]
    
    return model_name

In [ ]:
def eval_all_models(models_dir, dataset_name, test):
    model_filenames = glob.glob(models_dir + "/*{}*.model".format(dataset_name))
    return [eval_learned_models(lightgbm.Booster(model_file=mf), extract_model_name(mf), test) for mf in model_filenames]


In [ ]:
_ = eval_all_models(MODELS_DIR, DATASET_NAME, test)

In [ ]:
def load_attacked_dataset(dataset_name, budget):
    # load train/valid/test
    train_att, valid_att, test_att, _ = load_atk_train_valid_test("../data/{}/train_B{}.csv.bz2".format(dataset_name, budget), 
                                                           "../data/{}/valid_B{}.csv.bz2".format(dataset_name, budget), 
                                                           "../data/{}/test_B{}.csv.bz2".format(dataset_name, budget))

    test_groups = test_att['instance_id'].value_counts().sort_index().values
    test_att = test_att.iloc[:, 1:]

    valid_groups = valid_att['instance_id'].value_counts().sort_index().values
    valid_att = valid_att.iloc[:, 1:]

    train_groups = train_att['instance_id'].value_counts().sort_index().values
    train_att = train_att.iloc[:, 1:]
    
    return train_att, train_groups, valid_att, valid_groups, test_att, test_groups

In [ ]:
def load_attacked_datasets(dataset_name, budgets):
    att_datasets = {}
    for b in budgets:
        att_datasets[b] = load_attacked_dataset(dataset_name, b)
    
    return att_datasets

In [ ]:
att_datasets = load_attacked_datasets(DATASET_NAME, [20])

In [ ]:
def eval_all_models_under_attack(models_dir, dataset_name, test, test_groups, budget):
    
    model_filenames = glob.glob(models_dir + "/*{}*.model".format(dataset_name))
    
    return [eval_learned_models(lightgbm.Booster(model_file=mf), 
                                   extract_model_name(mf), 
                                   test, 
                                   test_groups, 
                                   budget=budget) for mf in model_filenames]
        
#         df_b = pd.concat([std_gbdt_df_att, 
#                           adv_boost_df_att_b5, 
#                           adv_boost_df_att_b15, 
#                           adv_boost_df_att_b150, 
#                           adv_boost_df_att_b300, 
#                           non_interf_df_att_b5,
#                           non_interf_df_att_b15,
#                           non_interf_df_att_b150,
#                           non_interf_df_att_b300
#                          ], 
#                          axis=0, 
#                          sort=False)

#         df_b.reset_index(inplace=True, drop=True)
        
#         dfs.append(df_b)
        
#     #df = functools.reduce(lambda left,right: pd.merge(left,right,on=['Model', 'Budget']), dfs)
#     df = pd.concat(dfs, axis=0, sort=False)
    
#     return df

In [ ]:
_ = eval_all_models_under_attack(MODELS_DIR, DATASET_NAME, att_datasets[20][4], att_datasets[20][5], 20)

In [ ]:
eval_under_attack_df

In [ ]:
overall_df = pd.concat([std_gbdt_df,
                        adv_boost_df_b5,
                        adv_boost_df_b15,
                        adv_boost_df_b150,
                        adv_boost_df_b300,
                        non_interf_df_b5,
                        non_interf_df_b15,
                        non_interf_df_b150,
                        non_interf_df_b300,
                        eval_under_attack_df], 
                       axis=0, 
                       sort=False)
overall_df.reset_index(inplace=True, drop=True)

In [ ]:
overall_df

# Save the DataFrame containing results

In [ ]:
overall_df.to_csv("../plots/plot_census.csv", sep=",", index=False)